In [53]:
import sqlite3 as db 
import pandas as pd
from queue import Queue
conn = db.connect('soccer/database.sqlite')

from sklearn.preprocessing import MinMaxScaler

home_adv_factor = 1
# #'attacking_work_rate', 'defensive_work_rate'
# attr_cols = ['overall_rating',
#        'potential', 'crossing', 'finishing', 'heading_accuracy',
#        'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
#        'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
#        'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
#        'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
#        'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
#        'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
#        'gk_reflexes']

attr_cols = ['crossing', 'finishing', 'heading_accuracy','short_passing', 'volleys', 'dribbling', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_positioning','gk_reflexes']

cols = ['Position']

field_pos = [['LB', 'LWB'], ['CB'], ['RB', 'RWB'], ['CM'], ['CAM', 'CDM'], ['CM'], ['LM', 'LW'], ['CF', 'ST'], ['RM', 'RW'],['GK']]    

missed_pos = {
    'CM': ['CAM','CDM'],
    'CAM': ['CM'],
    'CDM': ['CM'],
    'RM': ['LM','ST','CF'],
    'LM': ['RM','CF','ST'],
    'ST':['CF']   
}

pos_iter = {
    'CM': 0, 'CAM':0, 'CDM':0,'RM':0,'LM':0,'ST':0, 'CF':0
}

vs = [8,7,6,5,4,3,2,1,0]

def find_best_player(data, pos):
    visited = []
    queue = Queue()
    queue.put(pos)
    while not queue.empty():
        cur_pos = queue.get()
        alternates = missed_pos[pos]
        visited.append(cur_pos)
        
        all_alts = []
        for alt_pos in alternates:
            if alt_pos in visited:
                continue
            # check if there are rows 
            alt_player = data['Position']
            all_alts.append((alt_player,alt_player['overall_rating']))
            if len(alt_player > 0):
                return (alt_pos,alt_player)
            else:
                queue.put(alt_pos)
        
        #find the best alternate position
        best = max(all_alts,lambda x: x[1])
        #update back the index 
        if best:
            pos_iter[best['position']] += 1
    

In [54]:
norm_team_name = {}
norm_team_name['Man United'] = 'Manchester United'
norm_team_name['Man City'] = 'Manchester City'
norm_team_name['QPR'] = 'Queens Park Rangers'
norm_team_name['West Brom'] = 'West Bromwich Albion'
norm_team_name['West Ham'] = 'West Ham United'
norm_team_name['Bournemouth'] = 'AFC Bournemouth'
norm_team_name['Leicester'] = 'Leicester City'
norm_team_name['Spurs'] = 'Tottenham Hotspur'



In [55]:
def get_all_players_info(year):
    query_str = """select  p.player_name,pa2.* from player_attributes pa2, player p where p.player_api_id = pa2.player_api_id and
    (pa2.player_api_id, pa2.date) in (
    select z.player_api_id, z.date from (
    select pa.player_api_id, pa.date, min(abs(strftime('%%s',pa.date) - strftime('%%s','%(year)s-09-01 00:00:00'))) from player_attributes pa, player p 
                                    where pa.player_api_id = p.player_api_id
                                    and strftime('%%s',pa.date) > strftime('%%s','2014-05-14 00:00:00') 
                                    and pa.date like '%(year)s%%'
                                    group by pa.player_api_id ) z )"""%{'year':year}
    
    all_player_attrs = pd.read_sql_query(query_str,conn)
    return all_player_attrs


In [56]:
def norm_team_names(team_info):
    
    for key,value in norm_team_name.items():
        team_info.loc[lambda df: df['HomeTeam'] == key, 'HomeTeam'] = value
        team_info.loc[lambda df: df['AwayTeam'] == key, 'AwayTeam'] = value
    
    return team_info

In [57]:
def get_full_data(year):
    ptm = pd.read_csv('player_pos_year.csv', dtype = {'overall_rating':float} )
    ptm = ptm[ptm['Year'] == int(year) + 1]
    
    all_players_info  = get_all_players_info(year)
        
    joined = all_players_info.merge(ptm,on = "player_name" , how='inner')
    return joined


In [58]:
def create_team(data, name):
    
    full_team = data[data['Team'].str.startswith(name)]
    #group by player name to eliminate duplicates
    #print(full_team.index.get_level_values(0))
    
    full_team = full_team.sort_values(by = ['player_name','overall_rating']).drop_duplicates(subset = ['player_name'], keep = 'first')
#     full_team = full_team.groupby(['player_name'],as_index = False).agg({'overall_rating':'max'})
    team_agg = {}
    
    team_df = {}
    
    df_start = pd.DataFrame()
    
    #field_pos = [['LB', 'LWB'], ['CB'], ['RB', 'RWB'], ['CM'], ['CAM', 'CDM'], ['CM'], ['LM', 'LW'], ['CF', 'ST'], ['RM', 'RW']]    

    # pick central defenders 
    defs = full_team[full_team['Position'].isin(field_pos[1])].sort_values(by = ['overall_rating'], ascending = False).head(2)
    defs['pos_index'] = [1,2]
    df_start = df_start.append(defs)

    # pick right defender
    rdef = full_team[full_team['Position'].isin(field_pos[2])].sort_values(by = ['overall_rating'], ascending = False).head(1)
    rdef['pos_index'] = [3]
    df_start = df_start.append(rdef)

    # left defender
    ldef = full_team[full_team['Position'].isin(field_pos[0])].sort_values(by = ['overall_rating'], ascending = False).head(1)
    ldef['pos_index'] = [0]

    df_start = df_start.append(ldef)
                          
    # pick 1 strong CDM or CAM
    top_mid_loc = 0
    cm2 = full_team[full_team['Position'].isin(field_pos[4] )].sort_values(by = ['overall_rating'], ascending = False).head(1)
    
    
    
    
    if(len(cm2) == 0):
        cm22 = full_team[full_team['Position'].isin(field_pos[3] )].sort_values(by = ['overall_rating'], ascending = False).head(1)
        cm22['pos_index'] = 6
        top_mid_loc = top_mid_loc + 1
        df_start = df_start.append(cm22)
    else:
        cm2['pos_index'] = [6]
        df_start = df_start.append(cm2)         
    
  
    # pick 2 central midfielders 
    mids = pd.DataFrame()
    cm = full_team[full_team['Position'].isin(field_pos[3])].sort_values(by = ['overall_rating'], ascending = False).iloc[top_mid_loc:top_mid_loc+2]
    mids = mids.append(cm)

    if len(mids) < 2:
        extras = 2 - len(mids)
        cm_add = full_team[full_team['Position'].isin(field_pos[4])].sort_values(by = ['overall_rating'], ascending = False).iloc[1:1 + extras]
        mids = mids.append(cm_add)

    mids['pos_index'] = [4,5]    
    df_start = df_start.append(mids)

    
    # pick 2 wing midfielders 
    
    lw = full_team[full_team['Position'].isin(field_pos[6])].sort_values(by = ['overall_rating'], ascending = False).head(1)
    
    if len(lw) == 0:
        lw2 = full_team[full_team['Position'].isin(field_pos[8])].sort_values(by = ['overall_rating'], ascending = False).iloc[1]
        lw2['pos_index'] = 7
        df_start = df_start.append(lw2)
    else:
        lw['pos_index'] = [7]
        df_start = df_start.append(lw)
    
    rw = full_team[full_team['Position'].isin(field_pos[8])].sort_values(by = ['overall_rating'], ascending = False).head(1)
    rw['pos_index'] = [9]
    df_start = df_start.append(rw)
    
    # pick 1 forwards 
    cf = full_team[full_team['Position'].isin(field_pos[7])].sort_values(by = ['overall_rating'], ascending = False).head(1)
    cf['pos_index'] = [8]
    df_start = df_start.append(cf)
    
    #pick 1 goalkeeper
    gk = full_team[full_team['Position'].isin(field_pos[9])].sort_values(by = ['overall_rating'], ascending = False).head(1)
    gk['pos_index'] = [10]
    df_start = df_start.append(gk)

    return df_start

In [46]:
def agg_attrs_all(team):
    
    agg_attrs = {}
    team_atts = team[attr_cols]
    for key in team_atts.keys():
        if key.startswith('gk'):
            agg_attrs[key] = team.sort_values(by = [key], ascending = False)[key].head(2).mean()
        else:
            agg_attrs[key] = team.sort_values(by = [key], ascending = False)[key].head(10).mean()
        
    agg_attrs_df = pd.DataFrame([agg_attrs]).astype(float)
    return agg_attrs_df


def agg_attributes(team):
    
    team.loc[lambda df: df['Position'] == 'Goalkeeper', 'Position'] = 'Defender'
    
    g = team.groupby(['Position'] , as_index=False).mean()
    return g.transpose()

In [47]:
def get_fixtures(year):
    f = pd.read_csv('epl-results-19932018/EPL_Set.csv')
    f = norm_team_names(f)
    return f[f['Season'].str.startswith(year)]
    


In [48]:
def combine_metrics_field_pos(i,row,home_data,away_data,team_form):
   
    def update_form(home_team_name, away_team_name, result):
        if result == 1:
            team_form[home_team_name] += 2
            team_form[away_team_name] -= 2
        elif result == -1:
            team_form[home_team_name] -= 3
            team_form[away_team_name] += 3
        else:
            team_form[home_team_name] -= 1
            team_form[away_team_name] += 1
        
    norm_attrs = {}
# field_pos = [['LB', 'LWB'], ['CB'], ['RB', 'RWB'], ['CM'], ['CAM', 'CDM'], ['CM'],
#              ['RM', 'RW'], ['CF', 'ST'], ['LM', 'LW']]    

# vs = [8,7,6,5,4,3,2,1,0]
    # for each team, generate  0 - 8, each integer corresponds to field score
    
    home_team_df = home_data[0]
    away_team_df = away_data[0]
    
    # for each row of the team, norm_attrs['pos_index'] = player['overall_rating']
    
    for key,r in home_team_df.iterrows():
        #if not r['overall_rating']:
        #print(r)
        norm_attrs['h_%s'%r['pos_index']] = r['overall_rating']
        
        
    for key,r in away_team_df.iterrows():
        #if not r['overall_rating']:
        #print(r)
        norm_attrs['a_%s'%r['pos_index']] = r['overall_rating']
    
#     # sort by position 
        
#     for i in range(0,len(vs)):
        
#         home_pos = field_pos[i]
#         print(home_team_df['pos_index'])
    
#         norm_attrs['h%s'%i] = home_team_df[home_team_df['Position'].isin(home_pos)]['overall_rating']
        
#     for i in range(0,len(vs)):
        
#         away_pos = field_pos[i]
    
#         norm_attrs['a%s'%i] = away_team_df[away_team_df['Position'].isin(away_pos)]['overall_rating']
        
    home_attrs = home_data[1]
    away_attrs = away_data[1]
    
    norm_attrs['Home_Team'] = row['HomeTeam']
    norm_attrs['Away_Team'] = row['AwayTeam']
    norm_attrs['Result'] = 1 if row['FTHG'] - row['FTAG'] > 0  else (-1 if row['FTHG'] - row['FTAG'] < 0 else 0)
    norm_attrs['Game_Week'] = (i//10) + 1
    norm_attrs['FTHG'] = row['FTHG']
    norm_attrs['FTAG'] = row['FTAG']
    norm_attrs['Home_Form'] = team_form[row['HomeTeam']]
    norm_attrs['Away_Form'] = team_form[row['AwayTeam']]
                                        
    update_form(row['HomeTeam'],row['AwayTeam'],norm_attrs['Result'])
    
    norm_df = pd.DataFrame(norm_attrs, index = [str(i)])
    
    return norm_df 

In [49]:
def get_all_team_data(year):
    
    data = {}

    joined_data = get_full_data(year)

    f = get_fixtures(year)
    teams = f['HomeTeam'].unique() #get all unique teams

    for team in teams:
        team_data = create_team(joined_data, team)
        team_attr = agg_attrs_all(team_data)
        data[team] = (team_data, team_attr)
    return data

In [50]:
def construct_training_data(years):
    
    full_res_df = pd.DataFrame()
    
    for year in years:
        
        team_store = get_all_team_data(year)
        i = 0
        res_df = pd.DataFrame()
        f = get_fixtures(year)
        team_form = {name:60 for name in f['HomeTeam'].unique()}
        for key,row in f.iterrows():
            home = row['HomeTeam']
            away = row['AwayTeam']
            home_attrs = team_store[home]
            away_attrs = team_store[away]

            res_df = res_df.append(combine_metrics_field_pos(i,row,home_attrs,away_attrs,team_form))
            i = i + 1
        
        full_res_df = full_res_df.append(res_df)

    return full_res_df

In [51]:
tr_data = construct_training_data(['2014','2015','2016'])


IndexError: single positional indexer is out-of-bounds

In [ ]:
tr_data.to_csv('Full_Data_fieldpos_all.csv')